<a href="https://colab.research.google.com/github/Uday18github/Build-Real-time-Google-Play-store-data-analytics---python/blob/main/timeserieslinechart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Dash and necessary components if not already installed
!pip install dash dash-core-components dash-html-components plotly pandas pytz
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from dash import Dash, html, dcc
from datetime import datetime
import pytz

# ---- Sample Data (Replace with your own) ----
# Sample format: Date, App, Category, Installs, Reviews
data = {
    "Date": pd.date_range("2024-01-01", periods=6, freq='M').tolist() * 3,
    "App": ["CoolApp", "BeautyPlus", "BizTrack"] * 6,
    "Category": ["Entertainment", "Beauty", "Business"] * 6,
    "Installs": [1000, 1500, 2500, 4000, 5000, 6500] * 3,
    "Reviews": [600, 700, 800, 900, 1000, 1100] * 3
}
df = pd.DataFrame(data)

# ---- Apply Filters ----
df = df[
    df["App"].str[0].str.upper().isin([chr(c) for c in range(ord('A'), ord('X')) if chr(c) not in ['X', 'Y', 'Z']])
    & ~df["App"].str.contains("s", case=False)
    & df["Category"].str[0].isin(['E', 'C', 'B'])
    & (df["Reviews"] > 500)
]

# ---- Translate Category Labels ----
translation_map = {
    "Beauty": "सौंदर्य",       # Hindi
    "Business": "வணிகம்",      # Tamil
    "Dating": "Partnersuche"   # German
}
df["Category"] = df["Category"].replace(translation_map)

# ---- Group & Pivot ----
df["Date"] = pd.to_datetime(df["Date"])
grouped = df.groupby(["Date", "Category"])["Installs"].sum().reset_index()
pivot_df = grouped.pivot(index="Date", columns="Category", values="Installs").fillna(0)

# ---- Calculate Month-over-Month Growth ----
growth_df = pivot_df.pct_change().fillna(0)

# ---- Time Check ----
def is_between_6_and_9_ist():
    ist = pytz.timezone('Asia/Kolkata')
    now = datetime.now(ist)
    return 18 <= now.hour < 21

# ---- Plotting ----
def create_figure():
    fig = go.Figure()
    for category in pivot_df.columns:
        fig.add_trace(go.Scatter(
            x=pivot_df.index,
            y=pivot_df[category],
            mode="lines+markers",
            name=category
        ))

        # Highlight growth areas > 20%
        over_growth = growth_df[category] > 0.2
        for i in range(1, len(pivot_df)):
            if over_growth.iloc[i]:
                fig.add_shape(
                    type="rect",
                    x0=pivot_df.index[i - 1],
                    x1=pivot_df.index[i],
                    y0=0,
                    y1=pivot_df[category].iloc[i],
                    fillcolor="rgba(0,200,0,0.2)",
                    line=dict(width=0),
                    layer="below"
                )
    fig.update_layout(
        title="Install Trends Over Time by Category",
        xaxis_title="Date",
        yaxis_title="Total Installs",
        hovermode="x unified"
    )
    return fig

# ---- Dash App ----
app = Dash(__name__)
app.layout = html.Div([
    html.H1("App Install Growth Trends"),
    dcc.Graph(figure=create_figure()) if is_between_6_and_9_ist()
    else html.P("Graph is visible only between 6 PM and 9 PM IST."),
])

if __name__ == "__main__":
    # Replaced app.run_server with app.run
    app.run(debug=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


<ipython-input-1-6a7b948eeb77>:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "Date": pd.date_range("2024-01-01", periods=6, freq='M').tolist() * 3,


<IPython.core.display.Javascript object>